In [ ]:
import os
import pathlib
import numpy as np
import cv2
from tensorflow import keras
import tensorflow as tf
from keras import layers, preprocessing, callbacks, losses, utils, models
import sys
import PyQt5
from PyQt5.QtWidgets import QApplication, QLabel, QWidget, QVBoxLayout, QHBoxLayout, QGridLayout,QFrame
from PyQt5.QtGui import QPixmap, QPainter, QPen, QColor
from PyQt5.QtCore import Qt
import time
import glob
from collections import defaultdict

In [ ]:
# Détection automatique du GPU Apple (Metal)
devices = tf.config.list_physical_devices()
gpu_devices = [d for d in devices if d.device_type == "GPU"]
if gpu_devices:
    print("✅ GPU Apple détecté :", gpu_devices)
else:
    print("⚠️ Aucun GPU Apple détecté, utilisation du CPU.")

In [ ]:
# =========================
# 1) Parsing des fichiers
# =========================

def load_sequence_txt(path):
    """
    Lit un .txt où chaque ligne est une frame : "v1;v2;...;vN;"
    -> retourne un numpy array (T, D) où T = nb de frames, D = nb de features par frame
    """
    feats = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            parts = [p for p in line.strip().split(";") if p.strip() != ""]
            if not parts:
                continue
            vec = list(map(float, parts))
            feats.append(vec)
    X = np.array(feats, dtype=np.float32)  # (T, D)
    return X

In [ ]:
# =========================
# 2) Prétraitements
# =========================

def normalize_framewise(X):
    """
    Normalisation simple frame-par-frame :
    - centrage par la moyenne
    - mise à l’échelle par l’écart-type
    Remarque : si tes features sont des coordonnées articulaires (x,y,z) concaténées,
    tu peux remplacer par un centrage/scale géométrique (soustraire le 'poignet', etc.).
    """
    Xn = X.copy()
    mu = Xn.mean(axis=0, keepdims=True)      # (1, D)
    sigma = Xn.std(axis=0, keepdims=True) + 1e-8
    Xn = (Xn - mu) / sigma
    return Xn

def load_dataset_from_folder(seq_folder, max_len=200):
    """
    Charge toutes les séquences *.txt du dossier.
    Chaque fichier correspond à un seul geste, dont le label est le nom du fichier (sans extension).
    Normalise chaque séquence.
    Pad/tronque à max_len.
    Retourne X (N, max_len, D), y (N,), class_names
    """
    sequences = sorted(glob.glob(os.path.join(seq_folder, "*.txt")))
    X_list, y_list = [], []
    label_to_id = {}
    next_id = 0

    for path in sequences:
        label = os.path.splitext(os.path.basename(path))[0]
        X = load_sequence_txt(path)
        X = normalize_framewise(X)

        # padding/tronquage à max_len
        if len(X) >= max_len:
            seg = X[:max_len]
        else:
            pad = np.zeros((max_len - len(X), X.shape[1]), dtype=X.dtype)
            seg = np.vstack([X, pad])

        if label not in label_to_id:
            label_to_id[label] = next_id
            next_id += 1
        y = label_to_id[label]
        X_list.append(seg)
        y_list.append(y)

    if not X_list:
        raise RuntimeError("Aucun segment construit (vérifie les chemins et les fichiers).")

    X = np.stack(X_list, axis=0)          # (N, max_len, D)
    y = np.array(y_list, dtype=np.int64)  # (N,)
    id_to_label = {v:k for k,v in label_to_id.items()}
    class_names = [id_to_label[i] for i in range(len(id_to_label))]
    return X, y, class_names

In [ ]:
# =========================
# 4) Modèle (BiLSTM)
# =========================

def make_model(T, D, C):
    """
    T = longueur max (max_len)
    D = nb features par frame
    C = nb de classes
    """
    inp = layers.Input(shape=(T, D))
    x = layers.Masking(mask_value=0.0)(inp)
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
    x = layers.Bidirectional(layers.LSTM(64))(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    out = layers.Dense(C, activation='softmax', dtype='float32')(x)
    model = models.Model(inp, out)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
# =========================
# 5) Entraînement + test
# =========================

# Dossier où se trouvent les fichiers *.txt
seq_folder = "/Users/valentindaveau/Documents/UE_Vision/Coordonnees" 

# Construire le dataset
X, y, class_names = load_dataset_from_folder(seq_folder, max_len=200)
print("Dataset:", X.shape, y.shape, class_names)

# Split simple (train/val)
idx = np.arange(len(X))
np.random.shuffle(idx)
split = int(0.8 * len(X))
tr, va = idx[:split], idx[split:]
Xtr, ytr = X[tr], y[tr]
Xva, yva = X[va], y[va]

T, D = X.shape[1], X.shape[2]
C    = len(class_names)

model_path = "modele_gestes.keras"

if os.path.exists(model_path):
    print("✅ Modèle trouvé, chargement du modèle sauvegardé...")
    model = keras.models.load_model(model_path)
    class_names = np.load("class_names.npy", allow_pickle=True).tolist()
    print("✅ Noms de classes chargés :", class_names)
else:
    print("⚙️  Aucun modèle trouvé, entraînement en cours...")
    model = make_model(T, D, C)
    history = model.fit(Xtr, ytr, validation_data=(Xva, yva), epochs=20, batch_size=32)
    model.save(model_path)
    print(f"💾 Modèle sauvegardé sous : {model_path}")
    np.save("class_names.npy", class_names)
    print("💾 Noms de classes sauvegardés dans class_names.npy")



def normalize_landmarks(X20: np.ndarray) -> np.ndarray:
    """
    X20: (20,3) ordre dataset.
    - centre sur le poignet (point 0)
    - mise à l’échelle par une distance de référence stable (ex: WRIST→MIDDLE_MCP)
    """
    wrist = X20[0]
    Xc = X20 - wrist
    # distance de ref: WRIST (0) -> MIDDLE MCP (index MediaPipe 9, devenu X20[8] après mapping)
    scale = np.linalg.norm(Xc[8]) + 1e-8
    return Xc / scale

In [ ]:
# =========================
# 8) Inférence (sortie)
# =========================

def predict_sequence(model, path_txt, max_len=200):
    X = load_sequence_txt(path_txt)
    X = normalize_framewise(X)
    # simple agrégation : on prend toute la séquence tronquée/paddée
    if len(X) >= max_len:
        Xp = X[:max_len]
    else:
        pad = np.zeros((max_len - len(X), X.shape[1]), dtype=X.dtype)
        Xp = np.vstack([X, pad])
    Xp = np.expand_dims(Xp, axis=0)  # (1, T, D)
    prob = model.predict(Xp, verbose=0)[0]  # (C,)
    k = int(np.argmax(prob))
    return k, float(prob[k])

k, p = predict_sequence(model, "1.txt", max_len=200)
print(f"Prédiction pour 1.txt : {class_names[k]} (p={p:.2f})")